In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [3]:
test = pd.read_csv(TEST)
test_ids = test['id']

In [4]:
# Carga de datos ya procesados
X=np.load(PREPROCESSED_DATA+'binScale.npz')
train = X['arr_0']
label = X['arr_1']
test = X['arr_2']
X.close()

In [5]:
rf=RandomForestClassifier(n_jobs=4, random_state=25)
mlp=MLPClassifier(hidden_layer_sizes=(200,200), random_state=25, max_iter=1000)
svc=SVC(C=65, random_state=25)
gbc=GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, random_state=25)

lr=LogisticRegression(max_iter=1000)

estimators=[('rf',rf),('mlp',mlp),('SVC',svc),('gbc',gbc)]
model=StackingClassifier(estimators=estimators,final_estimator=lr,n_jobs=2,verbose=100)

In [6]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

[0.93972603 0.92657534 0.92931507 0.92657534 0.92986301]
0.9304109589041095


## Generar fichero de Kaggle

In [7]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try9-noW.csv", index=False)